In [1]:
import copy
import multiprocessing

import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from xgboost import XGBClassifier

def gridsearch(df: pd.DataFrame, random_state=1, also_random=True, max_depth=[1, 2, 3]) -> pd.DataFrame:
  X = df.drop(columns='target').values
  y = df['target'].values
  
  param_grid=dict(max_depth=max_depth, scale_pos_weight=[sum(~y)/sum(y), 1])
  
  X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=1/3, random_state=random_state)
  clf = XGBClassifier(random_state=random_state, tree_method='hist', n_jobs=10)
  gridsearch = GridSearchCV(
    clf, 
    param_grid=param_grid, 
    scoring='average_precision', 
    n_jobs=30,
    cv=StratifiedKFold(shuffle=True, random_state=random_state),
    return_train_score=True
  )
  
  if also_random: 
    gridsearch_random = copy.deepcopy(gridsearch)
    np.random.seed(random_state)
    y_random = copy.deepcopy(y_trainval)
    np.random.shuffle(y_random)
  
  gridsearch.fit(X_trainval, y_trainval)
  df_dict = dict(
      mean_train=gridsearch.cv_results_['mean_train_score'],
      std_train=gridsearch.cv_results_['std_train_score'],
      mean_test=gridsearch.cv_results_['mean_test_score'],
      std_test=gridsearch.cv_results_['std_test_score'],
      test_fold0=gridsearch.cv_results_[f'split0_test_score'],
      test_fold1=gridsearch.cv_results_[f'split1_test_score'],
      test_fold2=gridsearch.cv_results_[f'split2_test_score'],
      test_fold3=gridsearch.cv_results_[f'split3_test_score'],
      test_fold4=gridsearch.cv_results_[f'split4_test_score']
  )
  
  if also_random: 
    gridsearch_random.fit(X_trainval, y_random)
    df_dict['mean_train_random']=gridsearch_random.cv_results_['mean_train_score']
    df_dict['std_train_random']=gridsearch_random.cv_results_['std_train_score']
    df_dict['mean_test_random']=gridsearch_random.cv_results_['mean_test_score']
    df_dict['std_test_random']=gridsearch_random.cv_results_['std_test_score']
  df = pd.DataFrame(df_dict, index=pd.Index([(d['max_depth'], d['scale_pos_weight'] > 1) for d in gridsearch.cv_results_['params']], name=('max_depth', 'balanced')))
  df['diff_train_test'] = (df['mean_test'] - df['mean_train']).abs() / df['mean_test']
  df['rstd_test'] = df['std_test'] / df['mean_test']
  if also_random: df['test_over_random'] = df['mean_test'] - df['mean_test_random']
  return df.sort_values('mean_test', ascending=False)
def get_x_y(df: pd.DataFrame): return df.drop(columns='target').values, df['target'].values
def report_performance(df_train: pd.DataFrame, df_test=None, random_state=1, max_depth=1, tree_method='hist', balanced=True, n_jobs=multiprocessing.cpu_count()):
  X, y = get_x_y(df_train)
  if df_test is None: X, X_test, y, y_test = train_test_split(X, y, test_size=1/3, random_state=random_state)
  else: X_test, y_test = get_x_y(df_test)
  clf = XGBClassifier(max_depth=max_depth, n_jobs=128, tree_method=tree_method, scale_pos_weight=sum(~y)/sum(y) if balanced else 1 , random_state=random_state)
  clf.fit(X, y)
  y_pred = clf.predict_proba(X_test)[:,1]
  return dict(ap=average_precision_score(y_test, y_pred), roc=roc_auc_score(y_test, y_pred))

chosen_hp = dict()
results = dict()

## Askubuntu

In [2]:
dataset = 'askubuntu'
chosen_hp[dataset] = dict()
results[dataset] = dict()

### Random

In [3]:
random = gridsearch(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'))
random[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  diff_train_test  rstd_test  test_over_random  \
max_depth balanced                                                            
3         False      0.050200         0.825770   0.159691          0.049308   
2         False      0.041473         0.346244   0.120017          0.040560   
3         True       0.036390         0.318876   0.092383          0.035518   
2         True       0.029424         0.038861   0.047644          0.028534   
1         False      0.019488         0.050299   0.094515          0.018554   
          True       0.016752         0.025251   0.107087          0.015825   

                    test_fold0  test_fold1  test_fold2  test_fold3  test_fold4  
max_depth balanced                                                              
3         False       0.057978    0.055663    0.050619    0.035078    0.051662  
2         False       0.046468    0.043193    0.046345    0.034082    0.037276  
3         True        0.035754    0.032753    0.035488    0.035192    0.042764  
2         True        0.029342    0.028542    0.032157    0.028488    0.028590  
1         False       0.018621    0.022406    0.019953    0.016761    0.019701  
          True        0.017591    0.017891    0.016554    0.013365    0.018357

In [4]:
chosen_hp[dataset]['random'] = dict(max_depth=2, balanced=True)

results[dataset]['random'] = report_performance(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'), **chosen_hp[dataset]['random'])
print(results[dataset]['random'])

{'ap': 0.023293264618648724, 'roc': 0.8946992586126661}


### Temporal

In [5]:
temporal = gridsearch(pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'))
temporal[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  diff_train_test  rstd_test  test_over_random  \
max_depth balanced                                                            
3         False      0.039860         1.150436   0.232144          0.039328   
2         False      0.031763         0.404810   0.154870          0.031233   
3         True       0.028169         0.315953   0.255624          0.027639   
2         True       0.020791         0.038073   0.263838          0.020254   
1         False      0.013962         0.130814   0.219161          0.013397   
          True       0.012130         0.055315   0.170149          0.011586   

                    test_fold0  test_fold1  test_fold2  test_fold3  test_fold4  
max_depth balanced                                                              
3         False       0.025780    0.042280    0.033903    0.044582    0.052753  
2         False       0.026208    0.030563    0.027827    0.034271    0.039943  
3         True        0.018628    0.031371    0.020469    0.035430    0.034949  
2         True        0.014498    0.020978    0.015061    0.028702    0.024717  
1         False       0.010621    0.012798    0.011182    0.017749    0.017459  
          True        0.010161    0.011052    0.011742    0.016107    0.011589

In [6]:
chosen_hp[dataset]['temporal'] = dict(max_depth=2, balanced=True)
results[dataset]['temporal'] = report_performance(
  df_train=pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'), 
  df_test=pd.read_pickle(f'datasets/{dataset}/temporal/test/2/features.pkl'),
  **chosen_hp[dataset]['temporal']
)
print(results[dataset]['temporal'])

{'ap': 0.004643793569510117, 'roc': 0.8513835399254048}


## Condmat

In [7]:
dataset = 'condmat'
chosen_hp[dataset] = dict()
results[dataset] = dict()

### Random

In [8]:
random = gridsearch(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'))
random[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

,,mean_test,diff_train_test,rstd_test,test_over_random,test_fold0,test_fold1,test_fold2,test_fold3,test_fold4
max_depth,balanced,,,,,,,,,
3,False,0.012176,3.747138,0.352454,0.009270,0.019603,0.006619,0.010045,0.013162,0.011449
2,False,0.011711,1.439408,0.301330,0.008951,0.017680,0.006682,0.012222,0.010868,0.011102
1,False,0.011336,0.122145,0.430077,0.008822,0.019543,0.006487,0.014112,0.007421,0.009115
3,True,0.011166,2.408231,0.368321,0.008398,0.018464,0.006692,0.007830,0.011276,0.011570
2,True,0.010128,0.985273,0.375302,0.007355,0.016543,0.006186,0.011873,0.006675,0.009365
1,True,0.009457,0.146815,0.277128,0.006947,0.012661,0.006343,0.012234,0.006894,0.009152


In [9]:
chosen_hp[dataset]['random'] = dict(max_depth=1, balanced=False)

results[dataset]['random'] = report_performance(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'), **chosen_hp[dataset]['random'])
print(results[dataset]['random'])

{'ap': 0.01161405871585572, 'roc': 0.7167960845889996}


### Temporal

In [13]:
temporal = gridsearch(pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'))
temporal[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  diff_train_test  rstd_test  test_over_random  \
max_depth balanced                                                            
3         False      0.006513         8.906790   0.507119          0.005228   
1         False      0.005152         0.206178   0.622107          0.003848   
2         False      0.004359         3.991703   0.299936          0.003044   
          True       0.003938         2.986969   0.283034          0.002622   
1         True       0.003246         0.492874   0.194280          0.001938   
3         True       0.003240        12.136351   0.241537          0.001921   

                    test_fold0  test_fold1  test_fold2  test_fold3  test_fold4  
max_depth balanced                                                              
3         False       0.011085    0.009803    0.003712    0.002941    0.005025  
1         False       0.006089    0.002433    0.002779    0.003432    0.011026  
2         False       0.006444    0.003775    0.002997    0.003291    0.005288  
          True        0.003795    0.004030    0.002880    0.003000    0.005983  
1         True        0.003892    0.002504    0.002665    0.003107    0.004061  
3         True        0.003482    0.002391    0.002591    0.003137    0.004600

In [14]:
!pwd

/tmp/bruingjde/complexnetworks2020-experiment


In [15]:
chosen_hp[dataset]['temporal'] = dict(max_depth=1, balanced=False)
results[dataset]['temporal'] = report_performance(
  df_train=pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'), 
  df_test=pd.read_pickle(f'datasets/{dataset}/temporal/test/2/features.pkl'),
  **chosen_hp[dataset]['temporal']
)
print(results[dataset]['temporal'])

{'ap': 0.004799299901218471, 'roc': 0.7319341272805222}


## Digg

In [16]:
dataset = 'digg'
chosen_hp[dataset] = dict()
results[dataset] = dict()

### Random

In [17]:
random = gridsearch(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'))
random[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  diff_train_test  rstd_test  test_over_random  \
max_depth balanced                                                            
3         False      0.004987         6.313893   0.434023          0.004546   
2         False      0.004901         2.855505   0.272762          0.004473   
1         False      0.004518         0.782532   0.256848          0.004085   
          True       0.003971         0.071352   0.295038          0.003540   
3         True       0.003629         3.050770   0.234899          0.003205   
2         True       0.003612         0.909619   0.238104          0.003163   

                    test_fold0  test_fold1  test_fold2  test_fold3  test_fold4  
max_depth balanced                                                              
3         False       0.004867    0.003302    0.007528    0.001987    0.007249  
2         False       0.006072    0.004624    0.005558    0.002415    0.005837  
1         False       0.005743    0.004265    0.005505    0.002472    0.004603  
          True        0.006090    0.003579    0.003870    0.002492    0.003824  
3         True        0.004678    0.003747    0.004352    0.002359    0.003009  
2         True        0.004388    0.003409    0.004738    0.002347    0.003179

In [18]:
chosen_hp[dataset]['random'] = dict(max_depth=1, balanced=True)

results[dataset]['random'] = report_performance(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'), **chosen_hp[dataset]['random'])
print(results[dataset]['random'])

{'ap': 0.004337040791269717, 'roc': 0.8586802216000715}


### Temporal

In [19]:
temporal = gridsearch(pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'))
temporal[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  diff_train_test  rstd_test  test_over_random  \
max_depth balanced                                                            
1         False      0.003340         1.864113   0.586591          0.003125   
          True       0.002464         0.157004   0.304304          0.002256   
2         True       0.002315         1.221856   0.340775          0.002091   
          False      0.002054        10.037198   0.126380          0.001802   
3         False      0.001974        23.748900   0.252407          0.001693   
          True       0.001649         7.913410   0.131763          0.001379   

                    test_fold0  test_fold1  test_fold2  test_fold3  test_fold4  
max_depth balanced                                                              
1         False       0.001972    0.001686    0.003937    0.002178    0.006930  
          True        0.003052    0.001618    0.003412    0.001566    0.002675  
2         True        0.001571    0.001830    0.003790    0.001951    0.002433  
          False       0.001771    0.002059    0.002368    0.001755    0.002316  
3         False       0.001707    0.001771    0.001855    0.001584    0.002955  
          True        0.001705    0.001843    0.001444    0.001351    0.001904

In [20]:
chosen_hp[dataset]['temporal'] = dict(max_depth=1, balanced=True)
results[dataset]['temporal'] = report_performance(
  df_train=pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'), 
  df_test=pd.read_pickle(f'datasets/{dataset}/temporal/test/2/features.pkl'),
  **chosen_hp[dataset]['temporal']
)
print(results[dataset]['temporal'])

{'ap': 0.0014433791227711338, 'roc': 0.8353133183027801}


## Enron

In [21]:
dataset = 'enron'
chosen_hp[dataset] = dict()
results[dataset] = dict()

### Random

In [22]:
random = gridsearch(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'))
random[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  diff_train_test  rstd_test  test_over_random  \
max_depth balanced                                                            
3         False      0.077844         0.445972   0.145409          0.076589   
          True       0.042155         0.213018   0.191616          0.040891   
2         False      0.035862         0.241268   0.195357          0.034583   
          True       0.022814         0.141970   0.068906          0.021526   
1         False      0.017305         0.030621   0.118209          0.016025   
          True       0.013299         0.002559   0.106276          0.012012   

                    test_fold0  test_fold1  test_fold2  test_fold3  test_fold4  
max_depth balanced                                                              
3         False       0.075250    0.069090    0.097159    0.082592    0.065130  
          True        0.042543    0.031007    0.056090    0.041708    0.039428  
2         False       0.037135    0.027194    0.048338    0.033124    0.033518  
          True        0.024880    0.020727    0.024251    0.022684    0.021529  
1         False       0.018222    0.014126    0.020375    0.017233    0.016568  
          True        0.013753    0.011673    0.015660    0.012050    0.013360

In [23]:
chosen_hp[dataset]['random'] = dict(max_depth=1, balanced=False)

results[dataset]['random'] = report_performance(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'), **chosen_hp[dataset]['random'])
print(results[dataset]['random'])

{'ap': 0.015961239641945298, 'roc': 0.8706734068173684}


### Temporal

In [24]:
temporal = gridsearch(pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'))
temporal[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  diff_train_test  rstd_test  test_over_random  \
max_depth balanced                                                            
3         False      0.044061         1.089245   0.405172          0.043510   
2         False      0.026599         0.516816   0.376079          0.026016   
3         True       0.022385         0.573626   0.177820          0.021833   
2         True       0.012708         0.263628   0.082945          0.012138   
1         False      0.010403         0.072873   0.196721          0.009853   
          True       0.007225         0.000276   0.100433          0.006679   

                    test_fold0  test_fold1  test_fold2  test_fold3  test_fold4  
max_depth balanced                                                              
3         False       0.046569    0.077264    0.037936    0.031470    0.027067  
2         False       0.025339    0.044877    0.027025    0.015393    0.020360  
3         True        0.021136    0.023953    0.021428    0.016617    0.028789  
2         True        0.011507    0.013899    0.012009    0.014051    0.012073  
1         False       0.010352    0.011283    0.013665    0.007599    0.009115  
          True        0.008107    0.007296    0.007711    0.007045    0.005967

In [25]:
chosen_hp[dataset]['temporal'] = dict(max_depth=1, balanced=False)
results[dataset]['temporal'] = report_performance(
  df_train=pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'), 
  df_test=pd.read_pickle(f'datasets/{dataset}/temporal/test/2/features.pkl'),
  **chosen_hp[dataset]['temporal']
)
print(results[dataset]['temporal'])

{'ap': 0.011743071145453058, 'roc': 0.8742894406283527}


## Slashdot

In [26]:
dataset = 'slashdot'
chosen_hp[dataset] = dict()
results[dataset] = dict()

### Random

In [27]:
random = gridsearch(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'))
random[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

,,mean_test,diff_train_test,rstd_test,test_over_random,test_fold0,test_fold1,test_fold2,test_fold3,test_fold4
max_depth,balanced,,,,,,,,,
3,False,0.031692,2.194365,0.336949,0.031468,0.031094,0.031040,0.013888,0.047172,0.035265
2,False,0.026893,1.207152,0.320932,0.026661,0.031894,0.038854,0.013321,0.027883,0.022513
1,False,0.020870,0.343495,0.381570,0.020634,0.035039,0.021693,0.011140,0.016350,0.020128
3,True,0.018688,0.576111,0.203866,0.018450,0.021156,0.020347,0.011230,0.019223,0.021485
2,True,0.014801,0.330896,0.251286,0.014578,0.015893,0.019206,0.009123,0.017749,0.012036
1,True,0.012800,0.133195,0.247555,0.012565,0.011561,0.018384,0.008645,0.013030,0.012383


In [28]:
chosen_hp[dataset]['random'] = dict(max_depth=1, balanced=True)

results[dataset]['random'] = report_performance(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'), **chosen_hp[dataset]['random'])
print(results[dataset]['random'])

{'ap': 0.007561794372363436, 'roc': 0.9375479975589296}


### Temporal

In [29]:
temporal = gridsearch(pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'))
temporal[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

,,mean_test,diff_train_test,rstd_test,test_over_random,test_fold0,test_fold1,test_fold2,test_fold3,test_fold4
max_depth,balanced,,,,,,,,,
3,False,0.038462,2.407260,0.547677,0.038339,0.005802,0.026977,0.056453,0.064850,0.038229
2,False,0.037618,0.900996,0.438604,0.037489,0.007726,0.034056,0.052812,0.041394,0.052103
1,False,0.026668,0.278493,0.520683,0.026537,0.007517,0.015641,0.046372,0.035945,0.027866
3,True,0.019677,0.584677,0.740143,0.019547,0.003905,0.011064,0.034120,0.040134,0.009164
2,True,0.015039,0.206926,0.481105,0.014914,0.007860,0.012770,0.027747,0.017784,0.009035
1,True,0.011729,0.158469,0.293371,0.011594,0.005868,0.011232,0.016555,0.012757,0.012234


In [30]:
results[dataset]['temporal'] = report_performance(
  df_train=pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'), 
  df_test=pd.read_pickle(f'datasets/{dataset}/temporal/test/2/features.pkl'),
  max_depth=1, 
  balanced=True
)
print(results[dataset]['temporal'])

{'ap': 0.0021196115301011513, 'roc': 0.9271843440929457}


## SO

In [31]:
dataset = 'so'
chosen_hp[dataset] = dict()
results[dataset] = dict()

### Random

In [32]:
random = gridsearch(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'))
random[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  diff_train_test  rstd_test  test_over_random  \
max_depth balanced                                                            
3         False      0.005612         3.068285   0.290527          0.004733   
2         False      0.005092         0.946758   0.485350          0.004224   
3         True       0.004834         0.996598   0.310788          0.003926   
2         True       0.004339         0.336862   0.436250          0.003464   
1         True       0.003427         0.015251   0.216366          0.002581   
          False      0.003401         0.119680   0.232589          0.002548   

                    test_fold0  test_fold1  test_fold2  test_fold3  test_fold4  
max_depth balanced                                                              
3         False       0.007584    0.007002    0.006059    0.003619    0.003796  
2         False       0.009955    0.004627    0.003970    0.003292    0.003616  
3         True        0.007181    0.005821    0.004573    0.003199    0.003398  
2         True        0.008104    0.003728    0.003112    0.003415    0.003336  
1         True        0.004493    0.004139    0.002741    0.002742    0.003022  
          False       0.004870    0.003584    0.002960    0.002729    0.002862

In [33]:
chosen_hp[dataset]['random'] = dict(max_depth=1, balanced=True)

results[dataset]['random'] = report_performance(pd.read_pickle(f'datasets/{dataset}/random/2/features.pkl'), **chosen_hp[dataset]['random'])
print(results[dataset]['random'])

{'ap': 0.0029410861485296186, 'roc': 0.732642064218942}


### Temporal

In [34]:
temporal = gridsearch(pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'))
temporal[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  diff_train_test  rstd_test  test_over_random  \
max_depth balanced                                                            
3         True       0.005664         1.021339   0.649192          0.005170   
2         False      0.005657         1.128548   0.479451          0.005174   
3         False      0.005155         4.021934   0.122284          0.004671   
2         True       0.004233         0.188115   0.531275          0.003742   
1         True       0.002903         0.059349   0.321596          0.002466   
          False      0.002750         0.525722   0.319569          0.002308   

                    test_fold0  test_fold1  test_fold2  test_fold3  test_fold4  
max_depth balanced                                                              
3         True        0.001942    0.003472    0.002820    0.011161    0.008928  
2         False       0.003282    0.004414    0.002885    0.009718    0.007985  
3         False       0.004750    0.005473    0.004171    0.005391    0.005991  
2         True        0.002372    0.002382    0.005608    0.002778    0.008027  
1         True        0.001580    0.003039    0.004416    0.003093    0.002388  
          False       0.001613    0.003260    0.003954    0.003050    0.001873

In [35]:
chosen_hp[dataset]['temporal'] = dict(max_depth=1, balanced=True)
results[dataset]['temporal'] = report_performance(
  df_train=pd.read_pickle(f'datasets/{dataset}/temporal/train/2/features.pkl'), 
  df_test=pd.read_pickle(f'datasets/{dataset}/temporal/test/2/features.pkl'),
  **chosen_hp[dataset]['temporal']
)
print(results[dataset]['temporal'])

{'ap': 0.0013407589765506579, 'roc': 0.7243614208966747}


In [36]:
pd.DataFrame({dataset: {method: performance['ap'] for method, performance in result.items()}for dataset, result in results.items()}).T

,random,temporal
askubuntu,0.023293,0.004644
condmat,0.011614,0.004799
digg,0.004337,0.001443
enron,0.015961,0.011743
slashdot,0.007562,0.002120
so,0.002941,0.001341


In [37]:
pd.DataFrame({dataset: {method: performance['roc'] for method, performance in result.items()}for dataset, result in results.items()}).T

,random,temporal
askubuntu,0.894699,0.851384
condmat,0.716796,0.731934
digg,0.858680,0.835313
enron,0.870673,0.874289
slashdot,0.937548,0.927184
so,0.732642,0.724361
